In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import re
from pprint import pprint
import country_converter as coco
import datetime
# cc = coco.CountryConverter()
# from geopy.exc import GeocoderTimedOut 
# from geopy.geocoders import Nominatim 
from config import gkey

In [2]:
countries = 'https://api.covid19api.com/countries'

In [3]:
response_countries = requests.get(countries).json()
pprint(response_countries)

[{'Country': 'Heard and Mcdonald Islands',
  'ISO2': 'HM',
  'Slug': 'heard-and-mcdonald-islands'},
 {'Country': 'Iraq', 'ISO2': 'IQ', 'Slug': 'iraq'},
 {'Country': 'Mauritius', 'ISO2': 'MU', 'Slug': 'mauritius'},
 {'Country': 'Yemen', 'ISO2': 'YE', 'Slug': 'yemen'},
 {'Country': 'Mozambique', 'ISO2': 'MZ', 'Slug': 'mozambique'},
 {'Country': 'Guinea', 'ISO2': 'GN', 'Slug': 'guinea'},
 {'Country': 'Tajikistan', 'ISO2': 'TJ', 'Slug': 'tajikistan'},
 {'Country': 'Bahamas', 'ISO2': 'BS', 'Slug': 'bahamas'},
 {'Country': 'Bolivia', 'ISO2': 'BO', 'Slug': 'bolivia'},
 {'Country': 'Cook Islands', 'ISO2': 'CK', 'Slug': 'cook-islands'},
 {'Country': 'Cuba', 'ISO2': 'CU', 'Slug': 'cuba'},
 {'Country': 'Seychelles', 'ISO2': 'SC', 'Slug': 'seychelles'},
 {'Country': 'South Georgia and the South Sandwich Islands',
  'ISO2': 'GS',
  'Slug': 'south-georgia-and-the-south-sandwich-islands'},
 {'Country': 'Tuvalu', 'ISO2': 'TV', 'Slug': 'tuvalu'},
 {'Country': 'Denmark', 'ISO2': 'DK', 'Slug': 'denmark'}

In [4]:
len(response_countries)

248

In [5]:
name = []
slug = []
ISO2 = []
for i in range(len(response_countries)):
    name.append(response_countries[i]["Country"])
    slug.append(response_countries[i]["Slug"])
    ISO2.append(response_countries[i]["ISO2"])

countries_slug_df = pd.DataFrame({"Entity": name,
                                 "slug": slug,
                                 "ISO2": ISO2})
countries_slug_df.head()

,Entity,slug,ISO2
0,Heard and Mcdonald Islands,heard-and-mcdonald-islands,HM
1,Iraq,iraq,IQ
2,Mauritius,mauritius,MU
3,Yemen,yemen,YE
4,Mozambique,mozambique,MZ


In [6]:
countries_slug_df.sort_values(by = ["Entity"], inplace = True, ignore_index = True)
countries_slug_df.head()

,Entity,slug,ISO2
0,ALA Aland Islands,ala-aland-islands,AX
1,Afghanistan,afghanistan,AF
2,Albania,albania,AL
3,Algeria,algeria,DZ
4,American Samoa,american-samoa,AS


In [18]:
countries_slug_df.to_csv("countries_raw.csv")

In [7]:
len(countries_slug_df)

248

In [8]:
countries_slug_df.describe()

,Entity,slug,ISO2
count,248,248,248
unique,248,248,248
top,American Samoa,ireland,SV
freq,1,1,1


In [9]:
entity1= list(set(countries_slug_df["Entity"]))
standard_names = coco.convert(names=entity1, to='name_short')
standardized_names1_df = pd.DataFrame({"Entity" : entity1,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names1_df.head()

Netherlands Antilles not found in regex


,Entity,standard_names
0,ALA Aland Islands,Aland Islands
1,Afghanistan,Afghanistan
2,Albania,Albania
3,Algeria,Algeria
4,American Samoa,American Samoa


In [10]:
std_countries_slug_df = pd.merge(countries_slug_df, standardized_names1_df, how = "left", left_on='Entity', right_on='Entity')
std_countries_slug_df.sort_values(by = "Entity", inplace = True)
std_countries_slug_df.head()

,Entity,slug,ISO2,standard_names
0,ALA Aland Islands,ala-aland-islands,AX,Aland Islands
1,Afghanistan,afghanistan,AF,Afghanistan
2,Albania,albania,AL,Albania
3,Algeria,algeria,DZ,Algeria
4,American Samoa,american-samoa,AS,American Samoa


In [22]:
std_countries_slug_df.to_csv("countries_slug.csv")

store response_countries in to a df.
Import csv into a df.
parse the country name column to add hyphens replacing spaces.
do a lookup on response_countries df to get the slugs.
create for loop to check all the slugs and make separate api calls.

In [11]:
stringency_df = pd.read_csv("covid-stringency-index.csv")
stringency_df.head()

,Entity,Code,Date,stringency_index
0,Afghanistan,AFG,1/1/2020,0.0
1,Afghanistan,AFG,1/2/2020,0.0
2,Afghanistan,AFG,1/3/2020,0.0
3,Afghanistan,AFG,1/4/2020,0.0
4,Afghanistan,AFG,1/5/2020,0.0


In [12]:
len(stringency_df)

62265

In [13]:
stringency_df["Entity"].nunique()

181

In [14]:
stringency_df['month'] = pd.DatetimeIndex(stringency_df['Date']).month
stringency_df['year'] = pd.DatetimeIndex(stringency_df['Date']).year
stringency_df.head()

,Entity,Code,Date,stringency_index,month,year
0,Afghanistan,AFG,1/1/2020,0.0,1,2020
1,Afghanistan,AFG,1/2/2020,0.0,1,2020
2,Afghanistan,AFG,1/3/2020,0.0,1,2020
3,Afghanistan,AFG,1/4/2020,0.0,1,2020
4,Afghanistan,AFG,1/5/2020,0.0,1,2020


In [15]:
stringency_df.loc[stringency_df['Entity'] == "Timor", 'Entity'] = "Timor-leste"

In [16]:
stringency_df.describe()

,stringency_index,month,year
count,62265.000000,62265.000000,62265.0
mean,49.679238,6.182109,2020.0
std,29.292449,3.279415,0.0
min,0.000000,1.000000,2020.0
25%,25.930000,3.000000,2020.0
50%,54.630000,6.000000,2020.0
75%,74.070000,9.000000,2020.0
max,100.000000,12.000000,2020.0


In [18]:
stringency_df.loc[stringency_df['Entity'] == "Timor-leste"]

,Entity,Code,Date,stringency_index,month,year
55597,Timor-leste,TLS,1/1/2020,0.00,1,2020
55598,Timor-leste,TLS,1/2/2020,0.00,1,2020
55599,Timor-leste,TLS,1/3/2020,0.00,1,2020
55600,Timor-leste,TLS,1/4/2020,0.00,1,2020
55601,Timor-leste,TLS,1/5/2020,0.00,1,2020
...,...,...,...,...,...,...
55941,Timor-leste,TLS,12/10/2020,27.78,12,2020
55942,Timor-leste,TLS,12/11/2020,27.78,12,2020
55943,Timor-leste,TLS,12/12/2020,27.78,12,2020
55944,Timor-leste,TLS,12/13/2020,27.78,12,2020


In [19]:
entity2= list(set(stringency_df["Entity"]))
standard_names2 = coco.convert(names=entity2, to='name_short')
standardized_names2_df = pd.DataFrame({"Entity" : entity2,
                                      "standard_names": standard_names2}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names2_df.head(10)

,Entity,standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,Andorra,Andorra
4,Angola,Angola
5,Argentina,Argentina
6,Aruba,Aruba
7,Australia,Australia
8,Austria,Austria
9,Azerbaijan,Azerbaijan


In [21]:
cleaned_stringency_df = stringency_df.groupby(by = ["Entity", "Code", "month", "year"], as_index = False)["stringency_index"].mean()
cleaned_stringency_df.head(-10)

,Entity,Code,month,year,stringency_index
0,Afghanistan,AFG,1,2020,0.000000
1,Afghanistan,AFG,2,2020,2.010690
2,Afghanistan,AFG,3,2020,40.831935
3,Afghanistan,AFG,4,2020,80.740000
4,Afghanistan,AFG,5,2020,84.260000
...,...,...,...,...,...
2132,Zambia,ZMB,10,2020,45.818387
2133,Zambia,ZMB,11,2020,45.370000
2134,Zambia,ZMB,12,2020,45.370000
2135,Zimbabwe,ZWE,1,2020,1.254194


In [22]:
std_stringency_df = pd.merge(cleaned_stringency_df, standardized_names2_df, how = "left", left_on='Entity', right_on='Entity')
std_stringency_df.sort_values(by = "Entity", inplace = True)
std_stringency_df.head()

,Entity,Code,month,year,stringency_index,standard_names
0,Afghanistan,AFG,1,2020,0.000000,Afghanistan
10,Afghanistan,AFG,11,2020,7.591333,Afghanistan
9,Afghanistan,AFG,10,2020,16.460323,Afghanistan
7,Afghanistan,AFG,8,2020,55.408710,Afghanistan
6,Afghanistan,AFG,7,2020,78.700000,Afghanistan


In [23]:
std_stringency_df.to_csv("cleaned_stringency.csv")

In [24]:
merged_stringency_df =pd.merge(std_countries_slug_df, std_stringency_df, how = "inner", on = "standard_names")
merged_stringency_df = merged_stringency_df[merged_stringency_df["standard_names"] != "not found"]
merged_stringency_df.rename(columns = {"Entity_x": "Entity", "Code" : "ISO3"}, inplace=True)
merged_stringency_df = merged_stringency_df[["Entity" , "standard_names", "slug", "ISO2", "ISO3", "month", "year", "stringency_index"]]
merged_stringency_df.sort_values( by = ["standard_names", "year", "month"], inplace = True, ignore_index=True)
merged_stringency_df.head()

,Entity,standard_names,slug,ISO2,ISO3,month,year,stringency_index
0,Afghanistan,Afghanistan,afghanistan,AF,AFG,1,2020,0.000000
1,Afghanistan,Afghanistan,afghanistan,AF,AFG,2,2020,2.010690
2,Afghanistan,Afghanistan,afghanistan,AF,AFG,3,2020,40.831935
3,Afghanistan,Afghanistan,afghanistan,AF,AFG,4,2020,80.740000
4,Afghanistan,Afghanistan,afghanistan,AF,AFG,5,2020,84.260000


In [78]:
merged_stringency_df.to_csv("merged_stringency.csv")

In [25]:
merged_stringency_df["standard_names"].nunique()

181

In [26]:
std_countries_slug_df[["Entity"]][~std_countries_slug_df["standard_names"].isin(set(merged_stringency_df["standard_names"]))].drop_duplicates().reset_index(drop = True)

,Entity
0,ALA Aland Islands
1,American Samoa
2,Anguilla
3,Antarctica
4,Antigua and Barbuda
...,...
62,Tuvalu
63,US Minor Outlying Islands
64,"Virgin Islands, US"
65,Wallis and Futuna Islands


In [27]:
std_stringency_df[["Entity"]][~std_stringency_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

,Entity


In [28]:
country_names= list(set(merged_stringency_df['slug']))
country_names.sort()
country_names

['afghanistan',
 'albania',
 'algeria',
 'andorra',
 'angola',
 'argentina',
 'aruba',
 'australia',
 'austria',
 'azerbaijan',
 'bahrain',
 'bangladesh',
 'barbados',
 'belarus',
 'belgium',
 'belize',
 'benin',
 'bermuda',
 'bhutan',
 'bolivia',
 'bosnia-and-herzegovina',
 'botswana',
 'brazil',
 'brunei',
 'bulgaria',
 'burkina-faso',
 'burundi',
 'cambodia',
 'cameroon',
 'canada',
 'cape-verde',
 'central-african-republic',
 'chad',
 'chile',
 'china',
 'colombia',
 'congo-brazzaville',
 'congo-kinshasa',
 'costa-rica',
 'cote-divoire',
 'croatia',
 'cuba',
 'cyprus',
 'czech-republic',
 'denmark',
 'djibouti',
 'dominica',
 'dominican-republic',
 'ecuador',
 'egypt',
 'el-salvador',
 'eritrea',
 'estonia',
 'ethiopia',
 'faroe-islands',
 'fiji',
 'finland',
 'france',
 'gabon',
 'gambia',
 'georgia',
 'germany',
 'ghana',
 'greece',
 'greenland',
 'guam',
 'guatemala',
 'guinea',
 'guyana',
 'haiti',
 'honduras',
 'hong-kong-sar-china',
 'hungary',
 'iceland',
 'india',
 'indones

In [31]:
len(country_names)

181

In [30]:
country_std_names= list(set(merged_stringency_df['standard_names']))
country_std_names.sort()
country_std_names

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Argentina',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo Republic',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'DR Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faeroe Islands',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 '

In [32]:
# Target country
# target_country = "India"
lat = []
lng = []
contry_geo_name = []
# Build the endpoint URL
for country in range(len(country_std_names)):
    geo_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(country_std_names[country], gkey)
    geo_data = requests.get(geo_url).json()
    contry_geo_name.append(geo_data["results"][0]["address_components"][0]["long_name"])
    lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
    time.sleep(1)
# pprint(geo_data)

In [33]:
contry_coord = pd.DataFrame({"Entity": contry_geo_name,
                            "lat": lat,
                            "lng" : lng,
                            "standard_names" : country_std_names})
contry_coord.head()

,Entity,lat,lng,standard_names
0,Afghanistan,33.939110,67.709953,Afghanistan
1,Albania,41.153332,20.168331,Albania
2,Algeria,28.033886,1.659626,Algeria
3,Andorra,42.506285,1.521801,Andorra
4,Angola,-11.202692,17.873887,Angola


In [34]:
contry_coord.to_csv("contry_coord.csv")

In [94]:
base_url = 'https://api.covid19api.com/total/country/'
# https://api.covid19api.com/total/country/india/status/confirmed
entity = []
confirmed = []
deaths = []
recovered = []
active = []
date = []

In [114]:
for country in range(181):
    searc_url = base_url + country_names[country]
    covid_response = requests.get(searc_url).json()
    for i in range(len(covid_response)):
        entity.append(covid_response[i]["Country"])
        confirmed.append(covid_response[i]["Confirmed"])
        deaths.append(covid_response[i]["Deaths"])
        recovered.append(covid_response[i]["Recovered"])
        active.append(covid_response[i]["Active"])
        date.append(covid_response[i]["Date"])
    time.sleep(1)  

In [115]:
global_covid_df = pd.DataFrame({"Entity" : entity,
                               "Confirmed" : confirmed,
                               "Deaths" : deaths,
                               "Recovered" : recovered,
                               "Active" : active,
                               "Date" : date})
global_covid_df.sort_values(by = ["Entity", "Date"], inplace = True, ignore_index= True)
global_covid_df.head(-10)

,Entity,Confirmed,Deaths,Recovered,Active,Date
0,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
1,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
2,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
3,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
4,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
...,...,...,...,...,...,...
72863,Zimbabwe,10129,277,8643,1209,2020-12-02T00:00:00Z
72864,Zimbabwe,10424,280,8754,1390,2020-12-03T00:00:00Z
72865,Zimbabwe,10547,281,8802,1464,2020-12-04T00:00:00Z
72866,Zimbabwe,10617,291,8844,1482,2020-12-05T00:00:00Z


In [92]:
search_url = "https://api.covid19api.com/total/country/india"
covid_response = requests.get(search_url).json()
covid_response[0]["Date"]

'2020-01-22T00:00:00Z'

In [116]:
global_covid_df.to_csv("global_covid_stats.csv")

In [69]:
global_covid_df = pd.read_csv("global_covid_stats.csv")
global_covid_df.head()

,Unnamed: 0,Entity,Confirmed,Deaths,Recovered,Active,Date
0,0,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
1,1,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
2,2,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
3,3,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
4,4,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z


In [70]:
global_covid_df["Confirmed_daily"] =global_covid_df["Confirmed"].diff().fillna(0)
global_covid_df["Deaths_daily"] =global_covid_df["Deaths"].diff().fillna(0)
global_covid_df["Recovered_daily"] =global_covid_df["Recovered"].diff().fillna(0)
global_covid_df["Active_daily"] =global_covid_df["Active"].diff().fillna(0)
global_covid_df = global_covid_df[['Entity','Date', 'Confirmed_daily', 'Deaths_daily', 'Recovered_daily',
       'Active_daily']]
num = global_covid_df._get_numeric_data()
num[num < 0] = 0
global_covid_df.head(-10)

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
1,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
2,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
3,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
4,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
72863,Zimbabwe,2020-12-02T00:00:00Z,0.0,0.0,0.0,0.0
72864,Zimbabwe,2020-12-03T00:00:00Z,295.0,3.0,111.0,181.0
72865,Zimbabwe,2020-12-04T00:00:00Z,123.0,1.0,48.0,74.0
72866,Zimbabwe,2020-12-05T00:00:00Z,70.0,10.0,42.0,18.0


In [47]:
len(global_covid_df)

72878

In [71]:
global_covid_df.to_csv("global_covid_daily.csv")

In [72]:
global_covid_df['month'] = pd.DatetimeIndex(global_covid_df['Date']).month
global_covid_df['year'] = pd.DatetimeIndex(global_covid_df['Date']).year
global_covid_df.head()

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,month,year
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
1,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
2,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
3,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
4,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020


In [73]:
global_covid_monthly_df = global_covid_df.groupby(["Entity", "month", "year"], as_index = False).agg({"Confirmed_daily": "sum",
                                                                                   "Deaths_daily": "sum",
                                                                                   "Recovered_daily": "sum",
                                                                                   "Active_daily" : "sum"})
global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0
...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,628.0
2049,Zambia,11,2020,1215.0,8.0,1318.0,454.0
2050,Zambia,12,2020,809.0,12.0,637.0,372.0
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0


In [74]:
len(global_covid_monthly_df)

2063

In [75]:
std_global_covid_monthly_df = pd.merge(global_covid_monthly_df,standardized_names1_df, how = "left", on = "Entity")
std_global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,standard_names
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,Afghanistan
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,Afghanistan
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,Afghanistan
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,Afghanistan
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,Afghanistan
...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,628.0,Zambia
2049,Zambia,11,2020,1215.0,8.0,1318.0,454.0,Zambia
2050,Zambia,12,2020,809.0,12.0,637.0,372.0,Zambia
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0,Zimbabwe


In [80]:
std_global_covid_monthly_stringency_df = pd.merge(std_global_covid_monthly_df, merged_stringency_df , 
                                                  how = "left", on = ["standard_names", "month", "year"])
std_global_covid_monthly_stringency_df = std_global_covid_monthly_stringency_df[['standard_names', 'month', 'year',
        'Confirmed_daily', 'Deaths_daily', 'Recovered_daily', 'Active_daily', 'stringency_index']]
std_global_covid_monthly_stringency_df.head(-10)

,standard_names,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,stringency_index
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,0.000000
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,2.010690
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,40.831935
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,80.740000
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,84.260000
...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,628.0,45.818387
2049,Zambia,11,2020,1215.0,8.0,1318.0,454.0,45.370000
2050,Zambia,12,2020,809.0,12.0,637.0,372.0,45.370000
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0,1.254194


In [101]:
main_df = pd.merge(std_global_covid_monthly_stringency_df, contry_coord, how = "left", on = "standard_names")
main_df = main_df[['standard_names', 'month', 'year', 'Confirmed_daily', 'Deaths_daily',
       'Recovered_daily', 'Active_daily', 'stringency_index', 'lat','lng']]
main_df.rename(columns = {'Confirmed_daily' : "Confirmed", 'Deaths_daily' : "Deaths",
                           'Recovered_daily': "Recovered", 'Active_daily' : "Active"}, inplace = True)
main_df.head(-10)

,standard_names,month,year,Confirmed,Deaths,Recovered,Active,stringency_index,lat,lng
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,0.000000,33.939110,67.709953
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,2.010690,33.939110,67.709953
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,40.831935,33.939110,67.709953
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,80.740000,33.939110,67.709953
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,84.260000,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,628.0,45.818387,-13.133897,27.849332
2049,Zambia,11,2020,1215.0,8.0,1318.0,454.0,45.370000,-13.133897,27.849332
2050,Zambia,12,2020,809.0,12.0,637.0,372.0,45.370000,-13.133897,27.849332
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0,1.254194,-19.015438,29.154857


In [102]:
main_df.to_csv("main.csv")

In [104]:
main_stats_df = main_df.groupby("standard_names", as_index = False).agg({"Confirmed": "sum",
                                                                                   "Deaths": "sum",
                                                                                   "Recovered": "sum",
                                                                                   "Active" : "sum"})
main_stats_df.sort_values('Confirmed', inplace= True, ascending=False, ignore_index=True)

main_stats_df.head(10)

,standard_names,Confirmed,Deaths,Recovered,Active
0,United States,16973375.0,307429.0,6303715.0,16770801.0
1,India,9956557.0,144452.0,9489740.0,1069706.0
2,Brazil,7040608.0,183735.0,6239192.0,2049400.0
3,China,6490161.0,4746.0,89147.0,6457638.0
4,Russia,2708940.0,47994.0,2155842.0,677393.0
5,France,2539650.0,59857.0,186535.0,2320216.0
6,United Kingdom,2001391.0,849209.0,5198.0,2666440.0
7,Turkey,1928165.0,17121.0,1691113.0,1387694.0
8,Italy,1888292.0,66568.0,1176785.0,918745.0
9,Spain,1783696.0,50516.0,150376.0,1629869.0


In [107]:
main_stats_df[:20].to_csv("main_stats_2020.csv")